# TP3

## Imports

In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode
from timeout_decorator import timeout, TimeoutError
from threading import Timer
from yake import KeywordExtractor


In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

In [ ]:
DECADE = '1900'

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

## Choisir de la décennie

In [ ]:
# Exemple de fichiers
files[:5]
len(files)

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

## Vectoriser les documents à l'aide de TF-IDF

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

### Instancier le modèle TF-IDF avec ses arguments

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

### Construire la matrice de vecteurs à l'aide de la fonction `fit_transform`

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

### Imprimer le vecteur tf-IDF du premier document

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

## Comprendre les vecteurs et leurs "distances"

In [ ]:
cosine([1, 2, 3], [1, 2, 3])

In [ ]:
cosine([1, 2, 3], [1, 2, 2])

In [ ]:
cosine([1, 2, 3], [2, 2, 2])

### Tests sur nos documents

In [ ]:
tfidf_array = tfidf_vectors.toarray()

In [ ]:
# Vecteur du document 0
tfidf_array[0]

In [ ]:
# Vecteur du document 1
tfidf_array[1]

In [ ]:
cosine(tfidf_array[0], tfidf_array[1])

## Appliquer un algorithme de clustering sur les vecteurs TF-IDF des documents

Pour en savoir plus sur le KMeans clustering :
- https://medium.com/dataseries/k-means-clustering-explained-visually-in-5-minutes-b900cc69d175

### Définir un nombre de clusters

In [ ]:
N_CLUSTERS = 3

### Instancier le modèle K-Means et ses arguments

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

### Appliquer le clustering à l'aide de la fonction `fit_predict`

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

### visualiser le contenu des differents textes de chaque clusters 

In [ ]:
# Lecture du  contenu des clusters avec une limite de temps
def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# gestion  du  délai d'exécution
def timeout_handler():
    raise TimeoutError("Function execution timed out")

# Exploration des données brutes de chaque cluster
for cluster_label, files_in_cluster in clustering.items():
    print(f"Cluster {cluster_label} :")
    #pprint(files_in_cluster)
    print("\n")

   
    content_list = []
    for txt in files_in_cluster:
        timer = Timer(5, timeout_handler)  
        try:
            timer.start()
            content = read_file_content(os.path.join(data_path, txt))
            content_list.append(content)
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")
            continue
        finally:
            timer.cancel()

    # Afficher le contenu de chaque texte dans le cluster
    for txt in files_in_cluster:
        try:
            content = read_file_content(os.path.join(data_path, txt))
            print(f"Text in Cluster {cluster_label}:")
            print(content)
            print("\n")
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")

In [ ]:
# Stockage du contenu de chaque cluster dans une liste
all_cluster_content = []

# Exploration des données brutes de chaque cluster
for cluster_label, files_in_cluster in clustering.items():
   

    
    cluster_content = ""
    for txt in files_in_cluster:
        try:
            content = read_file_content(os.path.join(data_path, txt))
            cluster_content += content + "\n"  
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")

    # Ajouter le contenu du cluster au corpus global
    all_cluster_content.append(cluster_content)

# Afficher le contenu de chaque cluster
for cluster_label, cluster_content in zip(clustering.keys(), all_cluster_content):
    print(f"Cluster {cluster_label} - All Content:")
    print(cluster_content)
    print("\n")

    



### Extraction de keywords des differents clusters 

### Keywords pour chaque clusters 

In [ ]:

kw_extractor = KeywordExtractor()

all_cluster_keywords = []

for cluster_label, files_in_cluster in clustering.items():
  
    cluster_content = ""
    for txt in files_in_cluster:
        try:
            content = read_file_content(os.path.join(data_path, txt))
            cluster_content += content + "\n"  # Concaténer le contenu des textes avec un saut de ligne
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")

    # Extraction des mots-clés du contenu du cluster
    cluster_keywords = kw_extractor.extract_keywords(cluster_content)

   
    all_cluster_keywords.append(cluster_keywords)

# Affichage des mots-clés de chaque cluster
for cluster_label, cluster_keywords in zip(clustering.keys(), all_cluster_keywords):
    print(f"Cluster {cluster_label} - Keywords:")
    pprint(cluster_keywords)
    print("\n")


In [ ]:
kw_extractor = KeywordExtractor()

for cluster_label, files_in_cluster in clustering.items():
    

    # Stocker le contenu de chaque texte dans le cluster
    cluster_content = ""
    for txt in files_in_cluster:
        try:
            content = read_file_content(os.path.join(data_path, txt))
            cluster_content += content + "\n"  # Concaténer le contenu des textes avec un saut de ligne
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")

    # Extraire les mots-clés du contenu du cluster
    cluster_keywords = kw_extractor.extract_keywords(cluster_content)

    # Ajouter les mots-clés du cluster à la liste globale
    all_cluster_keywords.extend(cluster_keywords)

# Afficher les mots-clés de tous les clusters
print("Keywords in All Clusters:")
kept_keywords = []
for kw, score in all_cluster_keywords:
    words = kw.split()
    if len(words) == 2:
        kept_keywords.append(kw)

print(f"Keywords: {', '.join(kept_keywords)}...")


## Visualiser les clusters

### Analyse de nos clusters avec wordcloud

### definition de nos stopwords 

In [ ]:
# Liste de Stopwords enrichis (Idem que dans s1)
sw = stopwords.words("french")
sw += ["abord", "ailleurs", "ainsi", "alors", "après", "avant", "avoir", "bien", "bientôt",
    "car", "ce", "ceci", "cela", "ces", "cet", "cette", "comme", "contre", "dans",
    "depuis", "dire", "doit", "donc", "elle", "encore", "enfin", "ensuite", "entre", "être",
    "fait", "faire", "faut", "hormis", "ici", "il", "ils", "jusqu'à", "les", "leur", "là",
    "ma", "mais", "mes", "moins", "mon", "ne", "non", "nous", "on", "ou", "ou bien", "par",
    "parce que", "parfois", "pas", "pendant", "peut", "plus", "plutôt", "pour", "puis",
    "quand", "quant à", "sans", "sa", "se", "ses", "son", "sous", "tant pis", "tandis que",
    "trois", "tôt ou tard", "toutefois", "toutes", "tu", "tous", "trois", "trois", "trois",
    "voilà", "à", "très"]
sw = set(sw)

### Ecriture dans un fichier temporaire des differents clusters pour l'analyse 

In [ ]:
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)

# Exploration des données brutes de chaque cluster
for cluster_label, files_in_cluster in clustering.items():
  

    # Stocker le contenu de chaque texte dans le cluster
    cluster_content = ""
    for txt in files_in_cluster:
        try:
            content = read_file_content(os.path.join(data_path, txt))
            cluster_content += content + "\n"  # Concaténer le contenu des textes avec un saut de ligne
        except TimeoutError:
            print(f"Reading {txt} took too long. Skipping.")

    # Écrire le contenu du cluster dans un fichier temporaire
    with open(os.path.join(temp_path, f'cluster_{cluster_label}.txt'), 'w', encoding='utf-8') as f:
        f.write(cluster_content)

print("Content of all clusters has been written to temporary files.")

In [ ]:
# Chemin du fichier qui regroupera le contenu de tous les clusters
output_path = os.path.join(temp_path, 'cluster_2.txt')

# Imprimer le contenu du fichier et constater les "déchets"
with open(output_path, 'r', encoding='utf-8') as f:
    content = f.read()

print(content)

In [ ]:
import nltk

def clean_text(content, folder=None, stop_words=None):
    # Si le dossier n'est pas spécifié, utilisez le chemin par défaut
    folder = folder or '.'

    # Utilisez une liste de stop-words fournie ou une liste vide par défaut
    sw = stop_words or []

    if folder is None:
        input_path = "cluster_2.txt"
        output_path = "all_clusters_clean.txt"
    else:
        input_path = f"{folder}/cluster_2.txt"
        output_path = f"{folder}/all_clusters_clean.txt"

    # Créez le fichier de sortie pour écrire le texte nettoyé
    with open(output_path, "w", encoding='utf-8') as output:
        # Utilisez le contenu du cluster au lieu de lire depuis un fichier
        text = cluster_content

        # Tokenisez les mots en utilisant nltk
        words = nltk.wordpunct_tokenize(text)

        # Filtrez les mots en fonction des critères définis
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]

        # Joignez les mots filtrés pour former une chaîne
        kept_string = " ".join(kept)

        # Écrivez la chaîne nettoyée dans le fichier de sortie
        output.write(kept_string)

    return f'Output has been written in {output_path}!'


In [ ]:
clean_text(content, folder=temp_path)

In [ ]:
with open(os.path.join(temp_path, f'all_clusters_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

print(after[:1000]) 



In [ ]:
from collections import Counter

# Compter la fréquence des mots dans le texte nettoyé
frequencies = Counter(after.split())

# Afficher les 10 mots les plus fréquents
print(frequencies.most_common(10))

### Réduire les vecteurs à 2 dimensions à l'aide de l'algorithme PCA
Cette étape est nécessaire afin de visualiser les documents dans un espace 2D

https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

### Générer le plot

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
legend = plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

# Explorer chaque cluster
for cluster_label in set(clusters):
    cluster_points = reduced_vectors[clusters == cluster_label]
    
    # Afficher les points du cluster
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], s=50, label=f'Cluster {cluster_label}', alpha=0.5)

plt.title('Clusters and Centroids')
plt.show()

In [ ]:
for cluster_label in set(clusters):
    cluster_points = reduced_vectors[clusters == cluster_label]
    
    # Afficher les données brutes du cluster
    print(f"Cluster {cluster_label} - Raw Data:")
    print(cluster_points)
    print("\n")

### WORD_EMBEDDING 

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [ ]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [ ]:
type(bigram_phrases.vocab)

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [ ]:
len(bigram_phrases.vocab.keys())


Prenons une clé au hasard :

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

Le dictionnaire indique le score de cette coocurrence :

In [ ]:
bigram_phrases.vocab[key_]

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:


# Définir différentes valeurs pour window et min_count
window_values = [3, 5, 7]
min_count_values = [3, 5, 10]

# Des essais-erreurs
best_model = None
best_score = float('-0.01')  

for window_size in window_values:
    for min_count_value in min_count_values:
        print(f"Entraînement avec window={window_size}, min_count={min_count_value}")
        
        # Entraînement du modèle
        model = Word2Vec(
            corpus,
            vector_size=32,
            window=window_size,
            min_count=min_count_value,
            workers=4,
            epochs=5
        )

        score = len(model.wv.key_to_index)

        print(f"Score obtenu : {score}\n")

        # Comparaison avec le meilleur modèle précédent
        if score > best_score:
            best_score = score
            best_model = model

print("Entraînement terminé.")


### Sauver le modèle dans un fichier

In [ ]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [ ]:
model.wv["ministre"]

### Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("ministre", "roi")

### Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("ministre", topn=10)

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

### Execution de 06 exemples 

In [ ]:

similarity_example_1 = model.wv.similarity('paris', 'france')
print(f"Similarité entre 'paris' et 'france': {similarity_example_1}")


similarity_example_2 = model.wv.similarity('londres', 'royaume_uni')
print(f"Similarité entre 'londres' et 'royaume_uni': {similarity_example_2}")


similarity_example_3 = model.wv.similarity('belgique', 'france')
print(f"Similarité entre 'belgique' et 'france': {similarity_example_3}")


most_similar_example_1 = model.wv.most_similar("ministre", topn=3)
print(f"Mots similaires à 'ministre': {most_similar_example_1}")


most_similar_example_2 = model.wv.most_similar("parlement", topn=3)
print(f"Mots similaires à 'parlement': {most_similar_example_2}")


most_similar_example_3 = model.wv.most_similar("entreprise", topn=3)
print(f"Mots similaires à 'entreprise': {most_similar_example_3}")

